In [90]:
import os
from langchain_core.tools import tool, InjectedToolArg
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv
from typing import Annotated

In [91]:
load_dotenv()
api_key = os.getenv("EXCHANGE_RATE_API_KEY")
print(api_key)

d023e9fb25bd8e457933d21f


In [92]:

# Creating Tool :- 

@tool
def get_conversion_factor(base_currency : str, target_currency : str) -> float:
    """
    This function fetches the currency conversion factor between the given base currency and the target currency.
    """
    
    url = f'https://v6.exchangerate-api.com/v6/{api_key}/pair/{base_currency}/{target_currency}'
    
    response = requests.get(url)
    
    return response.json()

@tool 
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    This function converts the currency with the help of base_currency_value and the conversion rate given.
    """
    
    return base_currency_value * conversion_rate
    

In [93]:
get_conversion_factor.invoke({'base_currency' : 'USD', 'target_currency' : 'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1755734401,
 'time_last_update_utc': 'Thu, 21 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1755820801,
 'time_next_update_utc': 'Fri, 22 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.071}

In [94]:
convert.invoke({'base_currency_value':100, 'conversion_rate':87.071})

8707.1

In [95]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash")


In [96]:
# TOOL BINDING
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [97]:
messages = [HumanMessage("What is the conversion factor between USD and INR? Based on this can you convert 10 USD to INR")]

messages

[HumanMessage(content='What is the conversion factor between USD and INR? Based on this can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [98]:
ai_message = llm_with_tools.invoke(messages)

In [99]:
messages.append(ai_message)

In [100]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"base_currency": "USD", "target_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--666179a5-2460-402b-b50e-149189b9be79-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': '0bdd177d-0b9d-4e55-90f2-ac82c0917f3f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 123, 'total_tokens': 274, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 97}})

In [101]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': '0bdd177d-0b9d-4e55-90f2-ac82c0917f3f',
  'type': 'tool_call'}]

In [ ]:
import json 

for tool_call in ai_message.tool_calls:
    if(tool_call['name'] == 'get_conversion_factor'):
        tool_message1 = get_conversion_factor.invoke(tool_call)
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        
        messages.append(tool_message1)
    
    if(tool_call['name'] == 'convert'):
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)
messages.append(HumanMessage(content="Please provide the final answer now. And list down the name of tools that you used for finding the final answer"))

In [106]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR? Based on this can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"base_currency": "USD", "target_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--666179a5-2460-402b-b50e-149189b9be79-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': '0bdd177d-0b9d-4e55-90f2-ac82c0917f3f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 123, 'total_tokens': 274, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 97}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": 

In [107]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 87.071.\n\n10 USD is equal to 870.71 INR.\n\nI used the `get_conversion_factor` tool to find the conversion factor.'